In [1]:
import tejapi
import pandas as pd
import numpy as np
import plotly.graph_objects as go
tejapi.ApiConfig.api_key = 'Your Key'
tejapi.ApiConfig.ignoretz = True

In [93]:
#所有公司資料
database = tejapi.get('TWN/AIM1A',
                    mdate = {'gte':'2020-01-01'},
                    paginate = True,
                    opts = {'pivot':True, 'columns': ['coid','mdate','4151','R106','R502', 'R503', 'R509', 'R631', 'K632', 'R103']},
                    chinese_column_name = True)
database

,公司代碼,財報年月,普通股每股現金股利（盈餘及公積）,營業利益率,速動比率,利息支出率,借款依存度,季底收款天數,季底售貨天數,ROE(A)－稅後
None,,,,,,,,,,
0,000020,2020-06-01,NaN,37.98,326.30,0.00,0.0,NaN,NaN,22.70
1,000020,2020-12-01,5.91,34.70,297.35,0.00,0.0,NaN,NaN,40.35
2,000020,2021-06-01,NaN,30.89,463.43,0.00,0.0,NaN,NaN,13.79
3,000022,2020-06-01,NaN,NaN,608.76,-0.66,0.0,NaN,NaN,-20.27
4,000022,2020-12-01,NaN,NaN,524.88,-0.55,0.0,NaN,NaN,-41.24
...,...,...,...,...,...,...,...,...,...,...
13690,Z5389,2020-06-01,NaN,17.80,NaN,NaN,NaN,NaN,NaN,12.23
13691,Z5389,2020-09-01,NaN,3.31,NaN,NaN,NaN,NaN,NaN,3.15
13692,Z5389,2020-12-01,NaN,-0.29,NaN,NaN,NaN,NaN,NaN,-0.33


In [94]:
dps_date = database[database['財報年月'].dt.month == 12].groupby(by = '公司代碼').last()['財報年月'].mode().values[0]
dps_date

numpy.datetime64('2020-12-01T00:00:00.000000000')

In [95]:
dps_radar= database[database['財報年月'] == dps_date].loc[:,['公司代碼','普通股每股現金股利（盈餘及公積）']].reset_index(drop=True)
dps_radar

,公司代碼,普通股每股現金股利（盈餘及公積）
0,000020,5.91
1,000022,NaN
2,000024,NaN
3,000025,NaN
4,000026,NaN
...,...,...
2598,Z4008,NaN
2599,Z4541,NaN
2600,Z4898,NaN
2601,Z5060,NaN


In [96]:
#處理其他科目
database = database.drop(axis =1, columns = '普通股每股現金股利（盈餘及公積）')
com_date = database.groupby(by = '公司代碼').last()['財報年月'].mode().values[0]
adj_radar = database[database['財報年月'] == com_date].reset_index(drop=True)
adj_radar

,公司代碼,財報年月,營業利益率,速動比率,利息支出率,借款依存度,季底收款天數,季底售貨天數,ROE(A)－稅後
0,000020,2021-06-01,30.89,463.43,0.00,0.0,NaN,NaN,13.79
1,000022,2021-06-01,NaN,640.57,-0.36,0.0,NaN,NaN,-20.60
2,000024,2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,5.42
3,000025,2021-06-01,30.81,574.12,0.09,0.0,NaN,NaN,9.00
4,000026,2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,4.06
...,...,...,...,...,...,...,...,...,...
2526,Z4008,2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,9.69
2527,Z4541,2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,13.87
2528,Z4898,2021-06-01,9.80,NaN,NaN,NaN,NaN,NaN,2.43
2529,Z5060,2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,7.81


In [97]:
adj_radar = adj_radar.merge(dps_radar, on = '公司代碼')
adj_radar

,公司代碼,財報年月,營業利益率,速動比率,利息支出率,借款依存度,季底收款天數,季底售貨天數,ROE(A)－稅後,普通股每股現金股利（盈餘及公積）
0,000020,2021-06-01,30.89,463.43,0.00,0.0,NaN,NaN,13.79,5.91
1,000022,2021-06-01,NaN,640.57,-0.36,0.0,NaN,NaN,-20.60,NaN
2,000024,2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,5.42,NaN
3,000025,2021-06-01,30.81,574.12,0.09,0.0,NaN,NaN,9.00,NaN
4,000026,2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,4.06,NaN
...,...,...,...,...,...,...,...,...,...,...
2525,Z4008,2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,9.69,NaN
2526,Z4541,2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,13.87,NaN
2527,Z4898,2021-06-01,9.80,NaN,NaN,NaN,NaN,NaN,2.43,NaN
2528,Z5060,2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,7.81,NaN


In [98]:
ranked_radar = adj_radar.loc[:, ['普通股每股現金股利（盈餘及公積）', '營業利益率','速動比率','利息支出率','借款依存度','季底收款天數','季底售貨天數','ROE(A)－稅後']].rank(pct=True)
ranked_radar.insert(0, '公司代碼', adj_radar['公司代碼'])
ranked_radar

,公司代碼,普通股每股現金股利（盈餘及公積）,營業利益率,速動比率,利息支出率,借款依存度,季底收款天數,季底售貨天數,ROE(A)－稅後
0,000020,0.93483,0.934052,0.926563,0.209302,0.103095,NaN,NaN,0.913225
1,000022,NaN,NaN,0.952161,0.173173,0.103095,NaN,NaN,0.024226
2,000024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.577641
3,000025,NaN,0.933223,0.944188,0.266611,0.103095,NaN,NaN,0.783360
4,000026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.484512
...,...,...,...,...,...,...,...,...,...
2525,Z4008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.811358
2526,Z4541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.915806
2527,Z4898,NaN,0.589797,NaN,NaN,NaN,NaN,NaN,0.347299
2528,Z5060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.724186


In [99]:
ranked_radar = ranked_radar.fillna(0)
ranked_radar

,公司代碼,普通股每股現金股利（盈餘及公積）,營業利益率,速動比率,利息支出率,借款依存度,季底收款天數,季底售貨天數,ROE(A)－稅後
0,000020,0.93483,0.934052,0.926563,0.209302,0.103095,0.0,0.0,0.913225
1,000022,0.00000,0.000000,0.952161,0.173173,0.103095,0.0,0.0,0.024226
2,000024,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.577641
3,000025,0.00000,0.933223,0.944188,0.266611,0.103095,0.0,0.0,0.783360
4,000026,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.484512
...,...,...,...,...,...,...,...,...,...
2525,Z4008,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.811358
2526,Z4541,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.915806
2527,Z4898,0.00000,0.589797,0.000000,0.000000,0.000000,0.0,0.0,0.347299
2528,Z5060,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.724186


In [101]:
#選擇公司
firm_radar_list = ranked_radar[ranked_radar['公司代碼'] == '2330'].iloc[0, 1:].tolist()
firm_radar_list

[0.9777729823047043,
 0.9655744504355039,
 0.6731011330255979,
 0.3806063122923588,
 0.4960267670430782,
 0.30135786246167323,
 0.5006527415143603,
 0.9209690230341541]

In [77]:
#抓出同產業
firms = tejapi.get('TWN/AIND',
                  paginate = True,
                  opts = {'columns':['coid','tejind2_c']},
                  chinese_column_name = True)
firms

,公司簡稱,TEJ產業名
None,,
0,000116,M30A 證券
1,000930,M30A 證券
2,000960,M30A 證券
3,0009A0,M30A 證券
4,1101,M11A 水泥製造
...,...,...
3050,9962,M20A 金屬基本
3051,9965,M26C 貨運倉儲業
3052,L2301,M23D 電子零組件


In [78]:
industry = firms[firms['公司簡稱'] == '2330'].iloc[0,1]
industry

'M23G 半導體'

In [79]:
peers = firms[firms['TEJ產業名'] == industry]['公司簡稱'].tolist()
peers

['2303',
 '2311',
 '2325',
 '2329',
 '2330',
 '2337',
 '2338',
 '2339',
 '2342',
 '2344',
 '2351',
 '2363',
 '2369',
 '2379',
 '2388',
 '2401',
 '2408',
 '2436',
 '2441',
 '2449',
 '2451',
 '2454',
 '2455',
 '2458',
 '2473',
 '3006',
 '3014',
 '3016',
 '3034',
 '3035',
 '3041',
 '3053',
 '3063',
 '3072',
 '3079',
 '3081',
 '3087',
 '3094',
 '3105',
 '3122',
 '3126',
 '3133',
 '3135',
 '3137',
 '3141',
 '3150',
 '3169',
 '3185',
 '3186',
 '3222',
 '3227',
 '3228',
 '3257',
 '3259',
 '3260',
 '3264',
 '3265',
 '3268',
 '3270',
 '3271',
 '3288',
 '3291',
 '3298',
 '3304',
 '3317',
 '3328',
 '3342',
 '3347',
 '3365',
 '3372',
 '3374',
 '3381',
 '3387',
 '3397',
 '3408',
 '3411',
 '3438',
 '3442',
 '3443',
 '3470',
 '3474',
 '3475',
 '3482',
 '3506',
 '3527',
 '3529',
 '3530',
 '3532',
 '3534',
 '3538',
 '3545',
 '3553',
 '3556',
 '3559',
 '3567',
 '3571',
 '3572',
 '3574',
 '3582',
 '3588',
 '3589',
 '3592',
 '3598',
 '3613',
 '3614',
 '3620',
 '3635',
 '3637',
 '3638',
 '3661',
 '3688',
 

In [88]:
industry_radar = ranked_radar[ranked_radar['公司代碼'].isin(peers)].loc[:, ['普通股每股現金股利（盈餘及公積）', '營業利益率','速動比率','利息支出率','借款依存度','季底收款天數','季底售貨天數','ROE(A)－稅後']].mean()
industry_radar

普通股每股現金股利（盈餘及公積）    0.591755
營業利益率               0.670769
速動比率                0.694656
利息支出率               0.414177
借款依存度               0.339199
季底收款天數              0.464471
季底售貨天數              0.465104
ROE(A)－稅後           0.689607
dtype: float64

In [89]:
industry_radar_list = industry_radar.tolist()
industry_radar_list

[0.5917545265840479,
 0.6707686496415238,
 0.6946563455428334,
 0.41417744423350705,
 0.339198581984028,
 0.46447082994389144,
 0.46510423142276913,
 0.6896074454404478]

In [75]:
categories = ['現金股利','營業利益率','速動比率', '利息支出率','借款依存度', '收款天數', '售貨天數', 'ROE']

In [102]:
fig = go.Figure()

fig.add_trace(go.Scatterpolar( r= firm_radar_list,
                           theta= categories,
                            fill='toself',
                            name='公司'))
fig.add_trace(go.Scatterpolar( r=industry_radar_list,
                           theta=categories,
                            fill='toself',
                            name='產業'))

fig.show()